In [18]:
#Importo as bibliotecas
import pandas as pd
import requests
from bs4 import BeautifulSoup
import xlsxwriter
import os
import time


def scoresfixtures(link,ids):
    '''
    Description: This Class picks all the games that had in one season and combinate all links to one especific list
    
    Inputs:
        - link: The link of the main page that have all season games desired.
        - ids: The ID of the championship table
        
    Outputs:
        - especific list that has all the links os all matches of the season
    
    
    '''
    req = requests.get(link)
    if req.status_code == 200:
        content = req.content

    soup = BeautifulSoup(content, 'html.parser')
    tb = soup.find(id=ids)

    s1= []
    s2= []
    for i in tb.find_all("a"):
            s1.append(str(i))
            s2.append(str(i.get_text('href')))


    # Calling DataFrame constructor after zipping 
    # both lists, with columns specified 
    di = pd.DataFrame(list(zip(s1, s2)), 
                   columns =['Codes', 'ID']) 

    s4=[]
    for i in di["Codes"]:
        i = i.replace('<a href="','')
        i = i.replace('</a>','')
        s4.append(str(i))


    s5 = []

    for i in di['Codes']:
        if "matches" in i:
            s5.append(str(i))
        else:
            s5.append(0)

    s6 = []
    for i in di["Codes"]:
        if '<a href="/en/squads/' in i:
            i = i.replace('<a href="/en/squads/','')
            i = i[0:8]
            s6.append(str(i))
        else:
            s6.append(0)        

    # Calling DataFrame constructor after zipping 
    # both lists, with columns specified 
    da = pd.DataFrame(list(zip(s1, s2,s4,s5,s6)), 
                   columns =['CODES', 'ID','URL_FINAL','PARTIDAS_2019',"TEAM_CODE"])        

    s9 = []
    for i in da["URL_FINAL"]:
        if 'Match Report' in i:
            s9.append(str(i))
        else:
            pass
    return s9


In [5]:
scoresfixtures("https://fbref.com/en/comps/24/3320/schedule/2019-Serie-A-Scores-and-Fixtures","div_sched_ks_3320_1")

['/en/matches/e1867e7b/Sao-Paulo-Botafogo-RJ-April-27-2019-Serie-A">Match Report',
 '/en/matches/3b7c1663/Chapecoense-Internacional-April-27-2019-Serie-A">Match Report',
 '/en/matches/92511070/Atletico-Mineiro-Avai-April-27-2019-Serie-A">Match Report',
 '/en/matches/2b7cbdc9/Flamengo-Cruzeiro-April-27-2019-Serie-A">Match Report',
 '/en/matches/24ae5ae7/Gremio-Santos-April-28-2019-Serie-A">Match Report',
 '/en/matches/68e3e0d8/Ceara-CSA-April-28-2019-Serie-A">Match Report',
 '/en/matches/853f8904/Atletico-Paranaense-Vasco-da-Gama-April-28-2019-Serie-A">Match Report',
 '/en/matches/0da2aca1/Bahia-Corinthians-April-28-2019-Serie-A">Match Report',
 '/en/matches/e7c665b2/Palmeiras-Fortaleza-April-28-2019-Serie-A">Match Report',
 '/en/matches/2c5a9086/Fluminense-Goias-April-28-2019-Serie-A">Match Report',
 '/en/matches/fd884122/Internacional-Flamengo-May-1-2019-Serie-A">Match Report',
 '/en/matches/87cdcb36/Corinthians-Chapecoense-May-1-2019-Serie-A">Match Report',
 '/en/matches/7be3236f/CSA

In [19]:
def planilhas(url):
    '''
    Description: This function goes to de URL of the match and treat all data in order to append it in one single Dataframe.
    
    Input:
        - url: Url of the html page
        
    Output:
        - Dataframe treated from the match saved on my machine excel file
    
    '''
    #make the request
    pg = 'https://fbref.com'
    url_pg = pg+ url
    req = requests.get(url_pg)
    if req.status_code == 200:
        content = req.content
    #accessing data from site
    soup = BeautifulSoup(content, 'html.parser')

    table_geral = soup.find_all(class_ = "table_outer_container")
    table_time_1 = table_geral[0]
    table_time_2 = table_geral[2]
    table_time_3 = soup.find(class_='venuetime')

    #collecting data


    table_torcida = soup.find_all('div',class_ = "scorebox_meta")
    oi_torcida = str(table_torcida)


    #treating data
    toby = oi_torcida.split('<small>')
    torcida = str(toby[2])
    torcida = torcida.split('</small>')
    torcida = str(torcida[0])
    estadio = toby[4]
    estadio = estadio.split('</small>')
    estadio = str(estadio[0])    


    
    #collecting data
    data = table_time_3.get('data-venue-date')        

    
    #treating data
    nome = str(soup.title)
    nome = nome.replace(" ","_")
    nome = nome.replace("<title>","")
    nome = nome.replace(".","")
    nome_final = nome.split("Report")[0]


    #treating data

    nome_final = nome_final.split("_Match")
    nome_final = nome_final[0]    


    # STR transform and reading tables
    table_str_1 = str(table_time_1)
    table_str_2 = str(table_time_2)
    df_1 = pd.read_html(table_str_1)[0]
    df_2 = pd.read_html(table_str_2)[0]

    #treating data

    time = str(nome_final)
    time = time.replace("_"," ")
    time = time.split(" vs ")
    time_1 = str(time[0])
    time_2 = str(time[1])

    #Dtframe transforming
    df_1 = pd.DataFrame(df_1)
    df_1.columns = df_1.columns.droplevel()
    df_1['Time'] = str(time_1)
    df_1['Time_Adversario'] = str(time_2)
    df_1['Confronto'] = str(nome_final)
    df_1['Data'] = str(data)
    df_1['Estadio'] = str(estadio)
    df_1['Torcida'] = torcida    


    df_2 = pd.DataFrame(df_2)
    df_2.columns = df_2.columns.droplevel()
    df_2['Time'] = str(time_2)
    df_2['Time_Adversario'] = str(time_1)
    df_2['Confronto'] = str(nome_final)
    df_2['Data'] = str(data)
    df_2['Estadio'] = str(estadio)
    df_2['Torcida'] = torcida


    #APPENDING Dataframes
    df_3 = df_1.append(df_2)
    
    #save excel 
    writer = pd.ExcelWriter(nome_final+'.xlsx')
    df_3.to_excel(writer,"Estatisticas")
    writer.save()

In [20]:
def tratamento(nome): 
    '''
    Description: This function goes through all files of the directiory and joins all them in one single dataframe saving in
    excel sheet.
    
    Input:
        - Nome: Name that you want for your excel sheet
        
    Output:
        - Dataframe of all games save as excel sheet
    
    
    '''
    entries = os.listdir()

    base = {}

    base = pd.DataFrame(base)


    for i in entries:
        if "xlsx" in i:
            base = base.append(pd.read_excel(i))
            base = base.drop_duplicates()

        #escrevendo em excelfile
        else:
            pass
    writer = pd.ExcelWriter("C:/Users/mathe/Codes/Brasileirao_Udacity/"+nome+ ".xlsx")
    base.to_excel(writer,nome)
    writer.save()
    

In [21]:
start = time.time()

for url in scoresfixtures("https://fbref.com/en/comps/24/3320/schedule/2019-Serie-A-Scores-and-Fixtures","div_sched_ks_3320_1"):
    planilhas(url)
    

print('Duration: {} seconds'.format(time.time() - start))

Duration: 457.47057032585144 seconds


In [22]:
start = time.time()
tratamento("Brasileirao")

print('Duration: {} seconds'.format(time.time() - start))

Duration: 14.959064483642578 seconds
